In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world")
world = root_body(mechanism)

RigidBody: "world"

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("body2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
mechanism

Vertex: world (root)
  Vertex: body1, Edge: 1
    Vertex: body2, Edge: 2

In [6]:
x = MechanismState{Float64}(mechanism);

In [7]:
rand!(x)
x.v

DataStructures.OrderedDict{RigidBodyDynamics.Joint,Array{Float64,1}} with 2 entries:
  Joint "1": Quaternion f… => [0.022818052000514344,0.2779646042271158,0.202017…
  Joint "2": Revolute joi… => [0.1902561880514717]

In [8]:
num_positions(x)

8

In [9]:
num_velocities(x)

7

In [10]:
cache = MechanismStateCache(mechanism, x);

In [11]:
relative_transform(cache, joint2.frameAfter, body1.frame)

Transform3D from "after_2" to "body1":
rotation: 2.620696098641373 rad about [0.7333541839221726,-0.5808145806469791,-0.353335624911667], translation: Vec(0.9896402742464595,0.2763285417564447,0.4523510266667443)


In [12]:
center_of_mass(cache)

Point3D in "world": Vec(0.8256499280302081,2.0641190538116265,0.21220408040483357)

In [13]:
mass(mechanism)

1.0595147692458962

In [14]:
transform(cache, Point3D{Float64}(joint2.frameAfter), root_frame(mechanism))

Point3D in "world": Vec(-0.049098340407602,2.321078413339505,0.4999720488727556)

In [15]:
relative_twist(cache, body2, body1)

Twist of "after_2" w.r.t "after_1" in "world":
angular: Vec(-0.05256764014154662,-0.21704934407517737,-0.06318969965892501), linear: Vec(-0.03814964255998343,-0.02938486013008279,0.1326703773533929)

In [16]:
relative_twist(cache, body1, world) + relative_twist(cache, body2, body1) + relative_twist(cache, world, body2)

Twist of "world" w.r.t "world" in "world":
angular: Vec(0.0,0.0,0.0), linear: Vec(0.0,0.0,0.0)

In [17]:
geometric_jacobian(cache, path(mechanism, body2, world))

GeometricJacobian: body: "world", base: "after_2", expressed in "world":
6x7 Array{Float64,2}:
  0.276299  -0.527071  -0.297541  …  -0.0       -0.0       -0.0      
  1.14083   -0.741923  -0.295685     -0.0       -0.0       -0.0      
  0.33213   -0.414424   0.907766     -0.0       -0.0       -0.0      
  0.200517   0.549197   1.29779      -0.527071  -0.297541   0.796031 
  0.154449  -0.593905  -0.589563     -0.741923  -0.295685  -0.601765 
 -0.697325   0.36476    0.233341  …  -0.414424   0.907766   0.0649054

In [18]:
mass_matrix(cache)

7x7 Array{Float64,2}:
  2.34651      -0.463879     -1.07555      …   1.56978      -0.160805 
  0.604421      1.6864       -1.11546         -0.690309     -0.774056 
 -0.418474     -0.636257      2.03323         -2.77556e-17   0.564541 
 -2.08167e-17   0.241339     -1.56978          6.59195e-17   0.325254 
 -0.241339     -1.11022e-16   0.690309         0.0          -0.870217 
  1.56978      -0.690309     -8.32667e-17  …   1.05951      -0.594846 
 -0.160805     -0.774056      0.564541        -0.594846      0.0997681

In [19]:
A = momentum_matrix(cache)

MomentumMatrix expressed in "world":
6x7 Array{Float64,2}:
  0.730631    1.2171    -2.55106   …  -2.05173   -0.277243    0.691124
 -0.0786521   0.620377  -0.429593      0.861001  -0.122196    0.279547
  1.97421    -2.41493    0.846393     -0.39205    2.26731     0.127914
 -1.3214      0.67671   -0.621992      0.315249  -0.843407    0.386023
  0.873281   -0.23635   -0.960544      0.313282   0.637579   -0.373954
  0.117192    0.144821  -1.2772    …  -0.961792  -0.0687683   0.963355

In [20]:
A * velocity_vector(x)

Momentum expressed in "world":
angular: Vec(-0.2113821405507873,0.042205543930668915,1.201710110989827), linear: Vec(-0.10728123128410114,1.133841773966284,0.028319245061647846)

In [21]:
accel = SpatialAcceleration(joint2.frameAfter, world.frame, joint2.frameAfter, rand(Vec{3, Float64}), rand(Vec{3, Float64}))
transform(cache, accel, world.frame)

SpatialAcceleration of "after_2" w.r.t "world" in "world":
angular: Vec(-0.4645243632373102,-0.7827303054171955,-0.15127681443235014), linear: Vec(0.2894732150363979,-1.3234567018302648,1.6921518241787077)

In [22]:
bias_acceleration(cache, body2)

SpatialAcceleration of "after_2" w.r.t "world" in "world":
angular: Vec(-0.0695059477689119,0.009280857626813297,0.025943462281147512), linear: Vec(-0.025171139439521845,-0.03765307298590397,-9.567609205815074)

In [23]:
v̇ = [j::Joint => rand(num_velocities(j))::Vector{Float64} for j in joints(mechanism)]
τ = inverse_dynamics(cache, v̇)

7-element Array{Float64,1}:
 -1.79213  
 -1.38677  
 13.27     
 -3.73127  
  8.81549  
 -0.0858691
 -9.23028  

In [24]:
rand_chain_mechanism(Float64, QuaternionFloating, Revolute{Float64}, Revolute{Float64})

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body3, Edge: joint3

In [25]:
rand_tree_mechanism(Float64, [QuaternionFloating; [Revolute{Float64} for i = 1 : 10]]...)

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body3, Edge: joint3
        Vertex: body4, Edge: joint4
        Vertex: body8, Edge: joint8
  Vertex: body5, Edge: joint5
  Vertex: body6, Edge: joint6
    Vertex: body7, Edge: joint7
    Vertex: body9, Edge: joint9
      Vertex: body11, Edge: joint11
  Vertex: body10, Edge: joint10